In [1]:
import numpy as np
from scipy.integrate import solve_ivp

def round_sig(x, sig=3):
    x = np.array(x)
    def _round_scalar(val):
        if val == 0:
            return 0
        digits = sig - int(np.floor(np.log10(abs(val)))) - 1
        return np.round(val, digits)
    vfunc = np.vectorize(_round_scalar)
    return vfunc(x)

ODE_FUNCS = {}
TRAIN_INIT_FUNCS = {}
TEST_INIT_FUNCS = {}
MECH_DIM = {}
POST_FILTER_FUNCS = {}

def sample_theta_pool(n_total, log_min, log_max, dim, seed=0):
    np.random.seed(seed)
    theta_pool = 10 ** np.random.uniform(log_min, log_max, size=(n_total, dim))
    theta_pool = round_sig(theta_pool)
    return theta_pool

def split_theta_pool(theta_pool, n_train, n_val, n_test):
    idx = np.random.permutation(len(theta_pool))
    n_total = n_train + n_val + n_test
    idx = idx[:n_total]
    train_idx = idx[:n_train]
    val_idx = idx[n_train:n_train+n_val]
    test_idx = idx[n_train+n_val:]
    return theta_pool[train_idx], theta_pool[val_idx], theta_pool[test_idx]

def filter_theta_pool(theta_array, filter_func, ode_func, min_pass):
    passed = []
    for theta in theta_array:
        if filter_func(theta, ode_func):
            passed.append(theta)
            if len(passed) >= min_pass:
                break
    return np.array(passed)

def main_generate_all_mechanisms(mechanisms, n_train, n_val, n_test, seed=0):
    for mech in mechanisms:
        print(f"\n Generating data for {mech}")
        ode_func = ODE_FUNCS[mech]
        train_init_func = TRAIN_INIT_FUNCS[mech]
        test_init_func = TEST_INIT_FUNCS[mech]
        dim = MECH_DIM[mech]
        post_filter_func = POST_FILTER_FUNCS.get(mech, None)
        pool_size = 10 * (n_train + n_val + n_test) if post_filter_func else (n_train + n_val + n_test)
        theta_pool_raw = sample_theta_pool(pool_size, -5, 5, dim, seed=seed)
        if post_filter_func:
            print("  Filtering kinetic constants with mechanism-specific criterion...")
            theta_pool = filter_theta_pool(theta_pool_raw, post_filter_func, ode_func, n_train + n_val + n_test)
        else:
            theta_pool = theta_pool_raw
        theta_train, theta_val, theta_test = split_theta_pool(theta_pool, n_train, n_val, n_test)
        generate_dataset_given_theta(theta_train, ode_func, train_init_func, test_init_func, f"{mech}_train.npz", mode="train")
        generate_dataset_given_theta(theta_val, ode_func, train_init_func, test_init_func, f"{mech}_val.npz", mode="train")
        generate_dataset_given_theta(theta_test, ode_func, train_init_func, test_init_func, f"{mech}_test.npz", mode="test")

In [2]:
def ode_M1(t, y, theta):
    S, P, cat, catS = y
    k1, k_1, k2, k_2 = theta
    dS_dt = k_1 * catS - k1 * S * cat
    dP_dt = k2 * catS - k_2 * cat * P
    dcat_dt = (k_1 + k2) * catS - (k1 * S + k_2 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2) * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt]

def train_inits_M1():
    cat0s = np.random.uniform(0.01, 0.1, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0])
    return inits

def test_inits_M1():
    cat0s = [np.random.uniform(0.01, 0.02), np.random.uniform(0.045, 0.055), np.random.uniform(0.09, 0.10)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0])
    return inits

ODE_FUNCS["M1"] = ode_M1
TRAIN_INIT_FUNCS["M1"] = train_inits_M1
TEST_INIT_FUNCS["M1"] = test_inits_M1
MECH_DIM["M1"] = 4

In [5]:
def ode_M2(t, y, theta):
    S, P, cat, catS, cat2 = y
    k1, k_1, k2, k_2, k3, k_3 = theta
    dS_dt = k_1 * catS - k1 * S * cat
    dP_dt = k2 * catS - k_2 * cat * P
    dcat_dt = (k_1 + k2) * catS + 2 * k_3 * cat2 - (k1 * S + k_2 * P) * cat - 2 * k3 * cat * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2) * catS
    dcat2_dt = k3 * cat * cat - k_3 * cat2
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dcat2_dt]

def train_inits_M2():
    cat0s = np.random.uniform(0.01, 0.1, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M2():
    cat0s = [np.random.uniform(0.01, 0.02), np.random.uniform(0.045, 0.055), np.random.uniform(0.09, 0.10)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def sample_theta_pool_M2(n_total, log_min=-5, log_max=5, dim=6, seed=0):
    np.random.seed(seed)
    theta_pool = 10 ** np.random.uniform(log_min, log_max, size=(n_total, dim))
    theta_pool = np.round(theta_pool, 3)
    return theta_pool

def M2_cat2_post_filter(theta, ode_func):
    S0 = 1.0
    P0 = 0.0
    cat0 = 0.05
    y0 = [S0, P0, cat0, 0.0, 0.0]
    t_span = (0, 100)
    t_eval = np.linspace(0, 100, 100)
    sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
    if (not sol.success) or np.any(np.isnan(sol.y)):
        return False
    S_traj = sol.y[0]
    cat2_traj = sol.y[4]
    S_start = S_traj[0]
    S_end = S_traj[-1]
    S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
    window = np.where((S_conv >= 0.2) & (S_conv <= 0.5))[0]
    if len(window) == 0:
        return False
    cat2_max = np.max(cat2_traj[window])
    cat_total = cat0
    if cat2_max > 0.1 * cat_total:
        return True
    return False

ODE_FUNCS["M2"] = ode_M2
TRAIN_INIT_FUNCS["M2"] = train_inits_M2
TEST_INIT_FUNCS["M2"] = test_inits_M2
MECH_DIM["M2"] = 6
POST_FILTER_FUNCS["M2"] = M2_cat2_post_filter

In [7]:
def ode_M3(t, y, theta):
    S, P, cat, cat2, cat2S = y
    k1, k_1, k2, k_2, k3, k_3 = theta
    dS_dt = k_1 * cat2S - k1 * S * cat2
    dP_dt = k2 * cat2S - k_2 * cat2 * P
    dcat2_dt = (k_1 + k2) * cat2S + k3 * cat * cat - (k1 * S + k_2 * P + k_3) * cat2
    dcat2S_dt = (k1 * S + k_2 * P) * cat2 - (k_1 + k2) * cat2S
    dcat_dt = 2 * k_3 * cat2 - 2 * k3 * cat * cat

    return [dS_dt, dP_dt, dcat_dt, dcat2_dt, dcat2S_dt]

def train_inits_M3():
    cat0s = np.random.uniform(0.01, 0.1, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M3():
    cat0s = [np.random.uniform(0.01, 0.02), np.random.uniform(0.045, 0.055), np.random.uniform(0.09, 0.10)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M3_cat_post_filter(theta, ode_func):
    S0 = 1.0
    P0 = 0.0
    cat0 = 0.05
    y0 = [S0, P0, cat0, 0.0, 0.0]
    t_span = (0, 100)
    t_eval = np.linspace(0, 100, 100)
    sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
    if (not sol.success) or np.any(np.isnan(sol.y)):
        return False
    S_traj = sol.y[0]
    cat_traj = sol.y[2]
    S_start = S_traj[0]
    S_end = S_traj[-1]
    S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
    window = np.where(S_conv < 0.1)[0]
    if len(window) == 0:
        return False
    cat_min = np.min(cat_traj[window])
    cat_total = cat0
    if cat_min > 0.05 * cat_total:
        return True
    return False

ODE_FUNCS["M3"] = ode_M3
TRAIN_INIT_FUNCS["M3"] = train_inits_M3
TEST_INIT_FUNCS["M3"] = test_inits_M3
MECH_DIM["M3"] = 6
POST_FILTER_FUNCS["M3"] = M3_cat_post_filter

In [9]:
def ode_M4(t, y, theta):
    S, P, cat, catS, X = y
    k1, k_1, k2, k_2 = theta
    dS_dt = k_1 * X * catS - k1 * S * cat
    dP_dt = k2 * X * catS - k_2 * cat * P
    dcat_dt = (k_1 + k2) * X * catS - (k1 * S + k_2 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2) * X * catS
    dX_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2) * X * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dX_dt]

def train_inits_M4():
    cat0s = np.random.uniform(0.01, 0.1, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 1.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 1.0])
    return inits

def test_inits_M4():
    cat0s = [np.random.uniform(0.01, 0.02), np.random.uniform(0.045, 0.055), np.random.uniform(0.09, 0.10)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 1.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 1.0])
    return inits

ODE_FUNCS["M4"] = ode_M4
TRAIN_INIT_FUNCS["M4"] = train_inits_M4
TEST_INIT_FUNCS["M4"] = test_inits_M4
MECH_DIM["M4"] = 4  

In [11]:
def ode_M5(t, y, theta):
    S, P, cat, catS, catP = y
    k1, k_1, k2, k_2, k3, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat
    dP_dt    = k3 * catP - k_3 * cat * P
    dcat_dt  = k_1 * catS + k3 * catP - (k1 * S + k_3 * P) * cat
    dcatS_dt = (k1 * S + k_2 * catP) * cat - (k_1 + k2 * cat) * catS
    dcatP_dt = k2 * catS * cat + k_3 * P * cat - (k3 + k_2 * cat) * catP
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dcatP_dt]

def train_inits_M5():
    cat0s = np.random.uniform(0.01, 0.1, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M5():
    cat0s = [np.random.uniform(0.01, 0.02), np.random.uniform(0.045, 0.055), np.random.uniform(0.09, 0.10)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

ODE_FUNCS["M5"] = ode_M5
TRAIN_INIT_FUNCS["M5"] = train_inits_M5
TEST_INIT_FUNCS["M5"] = test_inits_M5
MECH_DIM["M5"] = 6  

In [13]:
def ode_M6(t, y, theta):
    S, P, cat, cat_star, cat_starS = y
    k1, k_1, k2, k_2, k3 = theta
    dS_dt = k_1 * cat_starS - k1 * S * cat_star
    dP_dt = k2 * cat_starS - k_2 * P * cat_star
    dcat_dt = -k3 * cat
    dcat_star_dt = k3 * cat + (k_1 + k2) * cat_starS - (k1 * S + k_2 * P) * cat_star
    dcat_starS_dt = (k1 * S + k_2 * P) * cat_star - (k_1 + k2) * cat_starS
    return [dS_dt, dP_dt, dcat_dt, dcat_star_dt, dcat_starS_dt]

def train_inits_M6():
    cat0s = np.random.uniform(0.03, 0.07, size=3)  # 3~7mol%
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M6():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M6_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_star_traj = sol.y[3]
        cat_starS_traj = sol.y[4]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_20 = np.argmin(np.abs(S_conv - 0.2))
        active_cat = cat_star_traj[idx_20] + cat_starS_traj[idx_20]
        if 0.1 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M6"] = ode_M6
TRAIN_INIT_FUNCS["M6"] = train_inits_M6
TEST_INIT_FUNCS["M6"] = test_inits_M6
MECH_DIM["M6"] = 5
POST_FILTER_FUNCS["M6"] = M6_activecat_post_filter

In [15]:
def ode_M7(t, y, theta):
    S, P, cat, catS, catS2 = y
    k1, k_1, k2, k_2, k3, k_3 = theta
    dS_dt    = k_1 * catS2 - k1 * S * catS + k_3 * catS - k3 * S * cat
    dP_dt    = k2 * catS2 - k_2 * catS * P
    dcat_dt  = k_3 * catS - k3 * S * cat
    dcatS_dt = k3 * S * cat - k_3 * catS + (k_1 + k2) * catS2 - (k1 * S + k_2 * P) * catS
    dcatS2_dt= (k1 * S + k_2 * P) * catS - (k_1 + k2) * catS2
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dcatS2_dt]

def train_inits_M7():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M7():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M7_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        catS_traj = sol.y[3]
        catS2_traj = sol.y[4]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_20 = np.argmin(np.abs(S_conv - 0.2))
        active_cat = catS_traj[idx_20] + catS2_traj[idx_20]
        if 0.1 * cat0 <= active_cat <= 0.8 * cat0:
            return True
    return False

ODE_FUNCS["M7"] = ode_M7
TRAIN_INIT_FUNCS["M7"] = train_inits_M7
TEST_INIT_FUNCS["M7"] = test_inits_M7
MECH_DIM["M7"] = 6
POST_FILTER_FUNCS["M7"] = M7_activecat_post_filter

In [17]:
def ode_M8(t, y, theta):
    S, P, cat, cat_star, cat_starS, L = y
    k1, k_1, k2, k_2, k3, k_3 = theta
    dS_dt      = k_1 * cat_starS - k1 * S * cat_star
    dP_dt      = k2 * cat_starS - k_2 * cat_star * P
    dcat_dt    = k_3 * L * cat_star - k3 * cat
    dcat_star_dt  = k3 * cat - k_3 * L * cat_star + (k_1 + k2) * cat_starS - (k1 * S + k_2 * P) * cat_star
    dcat_starS_dt = (k1 * S + k_2 * P) * cat_star - (k_1 + k2) * cat_starS
    dL_dt      = k3 * cat - k_3 * L * cat_star
    return [dS_dt, dP_dt, dcat_dt, dcat_star_dt, dcat_starS_dt, dL_dt]

def train_inits_M8():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0, 1.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0, 1.0])
    return inits

def test_inits_M8():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0, 1.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0, 1.0])
    return inits

def M8_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0, 1.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_star_traj = sol.y[3]
        cat_starS_traj = sol.y[4]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_star_traj[idx_50] + cat_starS_traj[idx_50]
        if 0.1 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M8"] = ode_M8
TRAIN_INIT_FUNCS["M8"] = train_inits_M8
TEST_INIT_FUNCS["M8"] = test_inits_M8
MECH_DIM["M8"] = 6
POST_FILTER_FUNCS["M8"] = M8_activecat_post_filter

In [19]:
def ode_M9(t, y, theta):
    S, P, cat, catS, inact_cat = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat
    dP_dt    = k2 * catS - k_2 * cat * P
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P + k_3) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2) * catS
    dinact_dt = k_3 * cat
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_dt]

def train_inits_M9():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M9():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M9_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M9"] = ode_M9
TRAIN_INIT_FUNCS["M9"] = train_inits_M9
TEST_INIT_FUNCS["M9"] = test_inits_M9
MECH_DIM["M9"] = 5
POST_FILTER_FUNCS["M9"] = M9_activecat_post_filter

In [21]:
def ode_M10(t, y, theta):
    S, P, cat, catS, inhibitor, inact_catI = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt      = k_1 * catS - k1 * S * cat
    dP_dt      = k2 * catS - k_2 * cat * P
    dcat_dt    = (k_1 + k2) * catS - (k1 * S + k_2 * P + k_3 * inhibitor) * cat
    dcatS_dt   = (k1 * S + k_2 * P) * cat - (k_1 + k2) * catS
    dinhibitor_dt = -k_3 * inhibitor * cat
    dinact_catI_dt = k_3 * inhibitor * cat
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinhibitor_dt, dinact_catI_dt]

def train_inits_M10():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 1.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 1.0, 0.0])
    return inits

def test_inits_M10():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 1.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 1.0, 0.0])
    return inits

def M10_inactcat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 1.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        inact_catI_traj = sol.y[5]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        inact_catI = inact_catI_traj[idx_50]
        if inact_catI >= 0.1 * cat0:
            return True
    return False

ODE_FUNCS["M10"] = ode_M10
TRAIN_INIT_FUNCS["M10"] = train_inits_M10
TEST_INIT_FUNCS["M10"] = test_inits_M10
MECH_DIM["M10"] = 5
POST_FILTER_FUNCS["M10"] = M10_inactcat_post_filter

In [23]:
def ode_M11(t, y, theta):
    S, P, cat, catS, inact_catS = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - (k1 + k_3) * S * cat
    dP_dt    = k2 * catS - k_2 * cat * P
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P + k_3 * S) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2) * catS
    dinact_catS_dt = k_3 * S * cat
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_catS_dt]

def train_inits_M11():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M11():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M11_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M11"] = ode_M11
TRAIN_INIT_FUNCS["M11"] = train_inits_M11
TEST_INIT_FUNCS["M11"] = test_inits_M11
MECH_DIM["M11"] = 5
POST_FILTER_FUNCS["M11"] = M11_activecat_post_filter

In [25]:
def ode_M12(t, y, theta):
    S, P, cat, catS, inact_catP = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat
    dP_dt    = k2 * catS - (k_2 + k_3) * P * cat
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P + k_3 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2) * catS
    dinact_catP_dt = k_3 * P * cat
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_catP_dt]

def train_inits_M12():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M12():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M12_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M12"] = ode_M12
TRAIN_INIT_FUNCS["M12"] = train_inits_M12
TEST_INIT_FUNCS["M12"] = test_inits_M12
MECH_DIM["M12"] = 5
POST_FILTER_FUNCS["M12"] = M12_activecat_post_filter

In [27]:
def ode_M13(t, y, theta):
    S, P, cat, catS, inact_cat2 = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt      = k_1 * catS - k1 * S * cat
    dP_dt      = k2 * catS - k_2 * cat * P
    dcat_dt    = (k_1 + k2) * catS - (k1 * S + k_2 * P + 2 * k_3 * cat) * cat
    dcatS_dt   = (k1 * S + k_2 * P) * cat - (k_1 + k2) * catS
    dinact_cat2_dt = k_3 * cat**2
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_cat2_dt]
def train_inits_M13():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M13():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M13_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.8 * cat0:
            return True
    return False

ODE_FUNCS["M13"] = ode_M13
TRAIN_INIT_FUNCS["M13"] = train_inits_M13
TEST_INIT_FUNCS["M13"] = test_inits_M13
MECH_DIM["M13"] = 5
POST_FILTER_FUNCS["M13"] = M13_activecat_post_filter

In [29]:
def ode_M14(t, y, theta):
    S, P, cat, catS, inact_catS = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat
    dP_dt    = k2 * catS - k_2 * cat * P
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2 + k_3) * catS
    dinact_catS_dt = k_3 * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_catS_dt]

def train_inits_M14():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M14():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M14_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M14"] = ode_M14
TRAIN_INIT_FUNCS["M14"] = train_inits_M14
TEST_INIT_FUNCS["M14"] = test_inits_M14
MECH_DIM["M14"] = 5
POST_FILTER_FUNCS["M14"] = M14_activecat_post_filter

In [31]:
def ode_M15(t, y, theta):
    S, P, cat, catS, inhibitor, inact_catSI = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat
    dP_dt    = k2 * catS - k_2 * cat * P
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2 + k_3 * inhibitor) * catS
    dinhibitor_dt = -k_3 * inhibitor * catS
    dinact_catSI_dt = k_3 * inhibitor * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinhibitor_dt, dinact_catSI_dt]

def train_inits_M15():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 1.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 1.0, 0.0])
    return inits

def test_inits_M15():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 1.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 1.0, 0.0])
    return inits

def M15_inactcat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 1.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        inact_catSI_traj = sol.y[5]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        inact_catSI = inact_catSI_traj[idx_50]
        if inact_catSI > 0.1 * cat0:
            return True
    return False

ODE_FUNCS["M15"] = ode_M15
TRAIN_INIT_FUNCS["M15"] = train_inits_M15
TEST_INIT_FUNCS["M15"] = test_inits_M15
MECH_DIM["M15"] = 5
POST_FILTER_FUNCS["M15"] = M15_inactcat_post_filter

In [33]:
def ode_M16(t, y, theta):
    S, P, cat, catS, inact_catS2 = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat - k_3 * S * catS
    dP_dt    = k2 * catS - k_2 * cat * P
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2 + k_3 * S) * catS
    dinact_catS2_dt = k_3 * S * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_catS2_dt]

def train_inits_M16():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M16():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M16_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M16"] = ode_M16
TRAIN_INIT_FUNCS["M16"] = train_inits_M16
TEST_INIT_FUNCS["M16"] = test_inits_M16
MECH_DIM["M16"] = 5
POST_FILTER_FUNCS["M16"] = M16_activecat_post_filter

In [35]:
def ode_M17(t, y, theta):
    S, P, cat, catS, inact_catSP = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat
    dP_dt    = k2 * catS - k_2 * cat * P - k_3 * P * catS
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2 + k_3 * P) * catS
    dinact_catSP_dt = k_3 * P * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_catSP_dt]

def train_inits_M17():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M17():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M17_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.9 * cat0:
            return True
    return False

ODE_FUNCS["M17"] = ode_M17
TRAIN_INIT_FUNCS["M17"] = train_inits_M17
TEST_INIT_FUNCS["M17"] = test_inits_M17
MECH_DIM["M17"] = 5
POST_FILTER_FUNCS["M17"] = M17_activecat_post_filter

In [37]:
def ode_M18(t, y, theta):
    S, P, cat, catS, inact_cat2S2 = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt    = k_1 * catS - k1 * S * cat
    dP_dt    = k2 * catS - k_2 * cat * P
    dcat_dt  = (k_1 + k2) * catS - (k1 * S + k_2 * P) * cat
    dcatS_dt = (k1 * S + k_2 * P) * cat - (k_1 + k2 + k_3 * catS) * catS
    dinact_cat2S2_dt = k_3 * catS ** 2
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_cat2S2_dt]

def train_inits_M18():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M18():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M18_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.8 * cat0:
            return True
    return False

ODE_FUNCS["M18"] = ode_M18
TRAIN_INIT_FUNCS["M18"] = train_inits_M18
TEST_INIT_FUNCS["M18"] = test_inits_M18
MECH_DIM["M18"] = 5
POST_FILTER_FUNCS["M18"] = M18_activecat_post_filter

In [39]:
def ode_M19(t, y, theta):
    S, P, cat, catS, inact_cat2S = y
    k1, k_1, k2, k_2, k_3 = theta
    dS_dt      = k_1 * catS - k1 * S * cat
    dP_dt      = k2 * catS - k_2 * cat * P
    dcat_dt    = (k_1 + k2) * catS - (k1 * S + k_2 * P + k_3 * catS) * cat
    dcatS_dt   = (k1 * S + k_2 * P) * cat - (k_1 + k2 + k_3 * cat) * catS
    dinact_cat2S_dt = k_3 * cat * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_cat2S_dt]

def train_inits_M19():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def test_inits_M19():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0])
    return inits

def M19_activecat_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        if 0.5 * cat0 <= active_cat <= 0.8 * cat0:
            return True
    return False

ODE_FUNCS["M19"] = ode_M19
TRAIN_INIT_FUNCS["M19"] = train_inits_M19
TEST_INIT_FUNCS["M19"] = test_inits_M19
MECH_DIM["M19"] = 5
POST_FILTER_FUNCS["M19"] = M19_activecat_post_filter

In [41]:
def ode_M20(t, y, theta):
    S, P, cat, catS, inact_cat, inact_catS = y
    k1, k_1, k2, k_2, k_3, k_4 = theta
    dS_dt      = k_1 * catS - k1 * S * cat
    dP_dt      = k2 * catS - k_2 * cat * P
    dcat_dt    = (k_1 + k2) * catS - (k1 * S + k_2 * P + k_3) * cat
    dcatS_dt   = (k1 * S + k_2 * P) * cat - (k_1 + k2 + k_4) * catS
    dinact_cat_dt = k_3 * cat
    dinact_catS_dt = k_4 * catS
    return [dS_dt, dP_dt, dcat_dt, dcatS_dt, dinact_cat_dt, dinact_catS_dt]

def train_inits_M20():
    cat0s = np.random.uniform(0.03, 0.07, size=3)
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0, 0.0])
    return inits

def test_inits_M20():
    cat0s = [np.random.uniform(0.03, 0.04), np.random.uniform(0.05, 0.06), np.random.uniform(0.06, 0.07)]
    S0_4 = np.round(np.random.uniform(0.4, 0.8), 4)
    P0_4 = 1.0 - S0_4
    pick4 = np.random.randint(0, 3)
    inits = []
    for i in range(3):
        S0 = 1.0
        P0 = 0.0
        cat0 = np.round(cat0s[i] * S0, 4)
        inits.append([S0, P0, cat0, 0.0, 0.0, 0.0])
    inits.append([S0_4, P0_4, np.round(cat0s[pick4] * S0_4, 4), 0.0, 0.0, 0.0])
    return inits

def M20_post_filter(theta, ode_func):
    for cat0 in [0.03, 0.05, 0.07]:
        y0 = [1.0, 0.0, cat0, 0.0, 0.0, 0.0]
        t_span = (0, 100)
        t_eval = np.linspace(0, 100, 100)
        sol = solve_ivp(lambda t, y: ode_func(t, y, theta), t_span, y0, t_eval=t_eval, method="LSODA")
        if (not sol.success) or np.any(np.isnan(sol.y)):
            continue
        S_traj = sol.y[0]
        cat_traj = sol.y[2]
        catS_traj = sol.y[3]
        inact_cat_traj = sol.y[4]
        inact_catS_traj = sol.y[5]
        S_start = S_traj[0]
        S_end = S_traj[-1]
        S_conv = (S_start - S_traj) / (S_start - S_end + 1e-10)
        idx_50 = np.argmin(np.abs(S_conv - 0.5))
        active_cat = cat_traj[idx_50] + catS_traj[idx_50]
        inact_cat = inact_cat_traj[idx_50]
        inact_catS = inact_catS_traj[idx_50]
        if (0.5 * cat0 <= active_cat <= 0.9 * cat0) and (inact_cat > 0.05 * cat0) and (inact_catS > 0.05 * cat0):
            return True
    return False

ODE_FUNCS["M20"] = ode_M20
TRAIN_INIT_FUNCS["M20"] = train_inits_M20
TEST_INIT_FUNCS["M20"] = test_inits_M20
MECH_DIM["M20"] = 6
POST_FILTER_FUNCS["M20"] = M20_post_filter

In [43]:
import os
import random
import pickle
import numpy as np
from scipy.integrate import solve_ivp

SIM_T_MAX = 100.0
SIM_N_POINTS = 100
N_TRAIN_PER_MECH = 2200
N_VAL_PER_MECH = 250
N_TEST_PER_MECH = 50
MECHANISMS = [f"M{i}" for i in range(1, 21)]
TRAIN_SAMPLE_POINTS = 20
SEED = 42

np.random.seed(SEED)
random.seed(SEED)

def simulate_trajectory(ode_func, theta, y0, n_points=SIM_N_POINTS):
    t_eval = np.linspace(0, SIM_T_MAX, n_points)
    sol = solve_ivp(lambda t, y: ode_func(t, y, theta), [0, SIM_T_MAX], y0,
                    t_eval=t_eval, method='LSODA')
    if not sol.success or np.any(np.isnan(sol.y)):
        raise RuntimeError('ODE solver failed')
    return sol.t, sol.y

def subsample_points(arr, t_full, n_points):
    idx = np.linspace(0, arr.shape[1] - 1, n_points).astype(int)
    return arr[:, idx], t_full[idx]

def extract_SPcat(traj):
    return traj[[0, 1, 2], :]

def generate_sample(theta, ode_func, init_func, train=True):
    inits = init_func()
    cat0s = [init[2] for init in inits]
    profiles = []
    t_subs = []
    for y0 in inits:
        t_full, traj = simulate_trajectory(ode_func, theta, y0)
        if train:
            sub, t_sub = subsample_points(traj, t_full, TRAIN_SAMPLE_POINTS)
        else:
            n_pts = random.randint(2, 20)
            sub, t_sub = subsample_points(traj, t_full, n_pts)

            noise_std = random.uniform(0.005, 0.05) * np.max(sub)
            sub = sub + np.random.normal(0.0, noise_std, size=sub.shape)
        profiles.append(extract_SPcat(sub).T)  # shape (n_pts, 3)
        t_subs.append(t_sub)

    min_len = min(p.shape[0] for p in profiles)
    profiles = [p[:min_len] for p in profiles]
    t_subs = [t[:min_len] for t in t_subs]
    sample_matrix = np.hstack(profiles)
    return np.array(cat0s), sample_matrix, t_subs[0]

def generate_dataset_given_theta(theta_array, ode_func, train_init, test_init, mode='train'):
    x1_list = []
    x2_list = []
    t_list = []
    for theta in theta_array:
        if mode == 'test':
            cat0, mat, t_sub = generate_sample(theta, ode_func, test_init, train=False)
        else:
            cat0, mat, t_sub = generate_sample(theta, ode_func, train_init, train=True)
        x1_list.append(cat0)
        x2_list.append(mat)
        t_list.append(t_sub)
    return np.array(x1_list), x2_list, t_list

def generate_simulation_dataset(output_dir='simulation_data'):
    os.makedirs(output_dir, exist_ok=True)
    x1_train_all, x1_val_all, x1_test_all = [], [], []
    x2_train_all, x2_val_all, x2_test_all = [], [], []
    t_train_all, t_val_all, t_test_all = [], [], []
    y_train_all, y_val_all, y_test_all = [], [], []
    for label, mech in enumerate(MECHANISMS):
        ode_func = ODE_FUNCS[mech]
        train_init_func = TRAIN_INIT_FUNCS[mech]
        test_init_func = TEST_INIT_FUNCS[mech]
        dim = MECH_DIM[mech]
        filter_func = POST_FILTER_FUNCS.get(mech)
        n_total = N_TRAIN_PER_MECH + N_VAL_PER_MECH + N_TEST_PER_MECH
        pool_size = 10 * n_total if filter_func else n_total
        theta_pool = sample_theta_pool(pool_size, -5, 5, dim, seed=SEED)
        if filter_func:
            theta_pool = filter_theta_pool(theta_pool, filter_func, ode_func, n_total)
        theta_train, theta_val, theta_test = split_theta_pool(
            theta_pool, N_TRAIN_PER_MECH, N_VAL_PER_MECH, N_TEST_PER_MECH)

        x1_train, x2_train, t_train = generate_dataset_given_theta(
            theta_train, ode_func, train_init_func, test_init_func, mode='train')
        x1_val, x2_val, t_val = generate_dataset_given_theta(
            theta_val, ode_func, train_init_func, test_init_func, mode='train')
        x1_test, x2_test, t_test = generate_dataset_given_theta(
            theta_test, ode_func, train_init_func, test_init_func, mode='test')

        x1_train_all.append(x1_train)
        x1_val_all.append(x1_val)
        x1_test_all.append(x1_test)
        x2_train_all.extend(x2_train)
        x2_val_all.extend(x2_val)
        x2_test_all.extend(x2_test)
        t_train_all.extend(t_train)
        t_val_all.extend(t_val)
        t_test_all.extend(t_test)
        y_train_all.append(np.full(len(x1_train), label))
        y_val_all.append(np.full(len(x1_val), label))
        y_test_all.append(np.full(len(x1_test), label))

    x1_train = np.concatenate(x1_train_all)
    x1_val = np.concatenate(x1_val_all)
    x1_test = np.concatenate(x1_test_all)
    y_train = np.concatenate(y_train_all)
    y_val = np.concatenate(y_val_all)
    y_test = np.concatenate(y_test_all)

    x2_train = np.stack(x2_train_all)
    x2_val = np.stack(x2_val_all)

    with open(os.path.join(output_dir, 'x1_train_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(x1_train, f)
    with open(os.path.join(output_dir, 'x1_val_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(x1_val, f)
    with open(os.path.join(output_dir, 'x1_test_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(x1_test, f)
    with open(os.path.join(output_dir, 'x2_train_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(x2_train, f)
    with open(os.path.join(output_dir, 'x2_val_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(x2_val, f)
    with open(os.path.join(output_dir, 'x2_test_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(x2_test_all, f)
    with open(os.path.join(output_dir, 'y_train_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(y_train, f)
    with open(os.path.join(output_dir, 'y_val_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(y_val, f)
    with open(os.path.join(output_dir, 'y_test_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(y_test, f)
    with open(os.path.join(output_dir, 't_train_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(t_train_all, f)
    with open(os.path.join(output_dir, 't_val_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(t_val_all, f)
    with open(os.path.join(output_dir, 't_test_M1_M20_train_val_test_set.pkl'), 'wb') as f:
        pickle.dump(t_test_all, f)
    print("Dataset generation complete.")

generate_simulation_dataset()

C:\Users\karl\anaconda3\envs\sbi-env\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
C:\Users\karl\anaconda3\envs\sbi-env\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in double_scalars
  
C:\Users\karl\anaconda3\envs\sbi-env\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in double_scalars
  
C:\Users\karl\anaconda3\envs\sbi-env\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\karl\anaconda3\envs\sbi-env\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\karl\anaconda3\envs\sbi-env\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\karl\anaconda3\envs\sbi-env\lib\site-packages\scipy\integrate\_ode.py:1353: UserWarning: lsoda: Excess accuracy requested (tolerances too smal

RuntimeError: ODE solver failed